In [1]:
import geopandas as gp

In [6]:
frsq_venues_path = '../data/frsq/frsq_extracted_venues.geojson'
FRSQ_VENUES = gp.read_file(frsq_venues_path)
opendc_segments_path = '../data/opendc_segments.geojson'
SEGMENTS = gp.read_file(opendc_segments_path)

In [39]:
def crs_prepossess(gpdf, init_crs, bfr_crs):
    gpdf_crs = gpdf.copy()
    if gpdf_crs.crs==None:
        gpdf_crs.crs = {'init': u'epsg:{}'.format(init_crs)}
    return gpdf_crs.to_crs(epsg=bfr_crs)

def bfr_20m(seg):
    return seg.geometry.buffer(20)

def get_objs_nearby(segments, objs, bfr_func, init_crs, bfr_crs):
    seg_crs = crs_prepossess(segments, init_crs, bfr_crs)
    obj_crs = crs_prepossess(objs, init_crs, bfr_crs)
    seg_crs.geometry = seg_crs.apply(bfr_20m,axis=1)

    sjoin = gp.tools.sjoin(seg_crs, obj_crs, how='left',op='intersects')
    obj_ids_nearby = set(sjoin[~sjoin.index_right.isnull()].index_right)
    objs_nearby = objs[objs.index.isin(obj_ids_nearby)]
    return objs_nearby


In [43]:
FRSQ_VENUES.shape, FRSQ_near_DCSEG.shape

((34060, 7), (20463, 7))

In [42]:
def write_geojson_nearby(objs_nearby_gpdf, objs_path, suffix='nearby'):
    import os
    fn, ext = os.path.splitext(objs_path)
    new_path = '{fn}_{sf}{ext}'.format(fn=fn, sf=suffix, ext=ext)
    with open(new_path, 'w') as f:
        f.write(objs_nearby_gpdf.to_json())
write_geojson_nearby(FRSQ_near_DCSEG, frsq_venues_path)